In [17]:
import pandas as pd
import numpy as np
from scipy.stats import norm

file_path = r'C:\Users\dylan\Desktop\QF609 (Risk Analytics)\Group Project 1\hist_data.xlsm'

# CONVERT SHEETS TO DATAFRAMES
excel_file = pd.ExcelFile(file_path)
sheet_names = excel_file.sheet_names
dfs = {sheet_name: excel_file.parse(sheet_name) for sheet_name in sheet_names}

print("Sheet Names:")
for sheet_name, df in dfs.items():
    print("    - ", sheet_name)

    # DECLARE VARIABLES
notional = 100_000_000
start_date = '2022-10-31'
end_date = '2024-10-30'
strike = 4.2/100
confidence_level = 0.95

Sheet Names:
    -  SofrCurve
    -  AAPL
    -  MSFT
    -  F
    -  BAC


In [16]:
## DATA PREP 1: SOFR 10Y TENOR
df = dfs['SofrCurve']
tenor_ten_years_df = df[df['Tenor'] == '10Y']
tenor_ten_years_df = tenor_ten_years_df.transpose()
tenor_ten_years_df = tenor_ten_years_df.iloc[2:]
tenor_ten_years_df = tenor_ten_years_df.reset_index()
tenor_ten_years_df = tenor_ten_years_df.rename(columns={'index':'date', 15:'sofr_rate'})

# DATA PREP 2: CAST DATE
df = tenor_ten_years_df
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# CALCULATE DAILY RETURNS
df['daily_return'] = df['sofr_rate'] - df['sofr_rate'].shift(1)
dfs['SofrCurve2'] = df

# MEAN & STD DEV OF RETURNS
mean_return = df['daily_return'].mean()
std_dev = df['daily_return'].std()

# Z SCORE BASED ON CONDFIDENCE LEVEL
z_score = norm.ppf(confidence_level)

# CALCULATE VAR
VAR = notional * z_score * std_dev

display(df.head())
print(f'SOFR Mean Daily Return is {mean_return*100:.5f}%')
print(f'SOFR Std Dev of Return is {std_dev:.5f}')
print(f'Parametric VAR for SOFR at {confidence_level * 100:.2f}% confidence level: ${VAR:,.2f}')

,sofr_rate,daily_return
date,,
2022-10-31,0.037353,NaN
2022-11-01,0.037209,-0.000144
2022-11-02,0.0373,0.000091
2022-11-03,0.037796,0.000495
2022-11-04,0.037988,0.000192


SOFR Mean Daily Return is 0.00267%
SOFR Std Dev of Return is 0.00071
Parametric VAR for SOFR at 95.00% confidence level: $115,987.90


In [15]:
def calculate_parametric_var(stock_symbol, confidence_level, notional, dfs):
    df = dfs[stock_symbol]

    # CALCULATE DAILY RETURNS
    df['daily_return'] = df['Adj Close'].pct_change()

    # MEAN & STD DEV OF RETURNS
    mean_return = df['daily_return'].mean()
    std_dev = df['daily_return'].std()

    # Z SCORE BASED ON CONFIDENCE LEVEL
    z_score = norm.ppf(confidence_level)

    # CALCULATE VAR
    VAR = notional * z_score * std_dev

    # display(df.head())
    print(f'{stock_symbol} Mean Daily Return is {mean_return * 100:.5f}%')
    print(f'{stock_symbol} Std Dev of Return is {std_dev:.5f}')
    print(f'Parametric {stock_symbol} VAR at {confidence_level * 100:.0f}% confidence level: ${VAR:,.2f}\n')

stocks_of_interest = ['AAPL', 'MSFT', 'F', 'BAC']
equity_dfs = {stock: dfs[stock] for stock in stocks_of_interest}

# Calculate parametric VAR for each stock
for i in equity_dfs:
    calculate_parametric_var(i, confidence_level, notional, dfs)

AAPL Mean Daily Return is 0.05680%
AAPL Std Dev of Return is 0.01587
Parametric AAPL VAR at 95% confidence level: $2,611,065.70

MSFT Mean Daily Return is 0.16922%
MSFT Std Dev of Return is 0.01788
Parametric MSFT VAR at 95% confidence level: $2,941,687.01

F Mean Daily Return is -0.06189%
F Std Dev of Return is 0.02254
Parametric F VAR at 95% confidence level: $3,707,195.13

BAC Mean Daily Return is -0.11051%
BAC Std Dev of Return is 0.01638
Parametric BAC VAR at 95% confidence level: $2,693,693.43

